In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az
import seaborn as sns

In [2]:
sns.set()

# Exercise session 6: WAIC

Let us test WAIC on the Beta-Binomial model: the *Hello World* probabilistic programming

Let us consider a coin that falls heads with probability $\theta=0.25$ (assumed unknown). 

Our prior knowledge of $\theta$ is encoded in a Beta distributions with parameters $a=20$ and $b=10$:

$$ p_{\rm prior}(\theta) = \frac{1}{B(a,b)} \theta^{a-1} (1-\theta)^{b-1}, \qquad \theta \in [0, 1],$$

where $B(a,b)$ is a proper normalization constant such that $\int_{0}^1 p_{\rm prior}(\theta) \; d \theta = 1.$ 

We toss the coin $n$ times and measure a number $y$ of heads. How does our belief of $\theta$ change with the measurement?

In [ ]:
a = 20 # prior: parameter a
b = 10 # prior: parameter b
n = 50 # likelihood: number of tosses
y = 15 # likelihood: number of HEADs observed. (Fixed in this example. It could be sampled form a binomial instead)

In [ ]:
with pm.Model() as beta_binomial:
    theta = pm.Beta("theta", alpha=a, beta=b)
    y_obs = pm.Binomial("y_obs", n=n, p=theta, observed=y)
    trace_bb = pm.sample(1000, random_seed=123, return_inferencedata=True)


In [ ]:
with beta_binomial:
    display(az.summary(trace_bb))

In [ ]:
az.plot_trace(trace_bb);

In [ ]:
az.plot_posterior(trace_bb, color="b");

## WAIC Definition

Let us denote by $\Theta$ the samples $\theta_0, \theta_1, \dots, \theta_{S-1}$ in the trace and by $y$ the measurement(s). 

* The log-pointwise-predictive-density $\rm lppd$ is defined as:

$$ {\rm lppd}(y, \Theta) = \sum_i \log \frac{1}{S} \sum_s p(y_i | \Theta_s),$$

* The effective number of parameters $p_{{\rm waic}}$ is defined as:

$$ p_{{\rm waic}} = \sum_i {\rm var}_{\theta} \log p(y_i|\theta)$$

* The WAIC criterion (the lower, the better) is:

$${\rm WAIC}(y, \Theta) = -2(\rm{lppd} - p_{\rm waic}).$$

* The expected log pointwise predictive density $\rm{elpd}$ (the higher, the better) is defined as:

$$\rm{elpd} = (\rm{lppd} - p_{\rm waic}).$$

## WAIC computation details

Let us consider a scalar observation $y$ (as in our example). In the definitions above, the outer summations on $i$ disappear. 

Let us denote by ``lik_vec`` and ``loglik_vec`` vectors containing, respectively, the likelihood and log-likelihood of $y$, for the samples of $\theta_s \in \Theta$:

$$\text{loglik_vec} = [\log p(y | \theta_0)\;\; \log p(y | \theta_1),\;\; \dots,\;\; \log p(y | \theta_{S-1})]^\top$$
$$\text{lik_vec} = [p(y | \theta_0)\;\; p(y | \theta_1),\;\; \dots,\;\; p(y | \theta_{S-1})]^\top$$

* $p_{{\rm waic}}$ can be computed as the sample variance of ``loglik_vec``. In Python:

``p_waic = np.var(loglik_vec)``

* ${\rm lppd}$ could be computed (in principle) as the logarithm of the average of ``lik_vec``. In Python:

``lppd = np.log(np.mean(lik_vec))``

* For better numerical precision, ${\rm lppd}$ computation is typically worked out in the log-domain, exploiting the logsumexp trick:

``lppd = scipy.special.logsumexp(log_lik_vec) - np.log(S)``

This exploits the identity:

$$\log {{\sum_s p(y_i | \theta_s)}} =  \log\left(\sum_s e^{\log p(y_i | \theta_s)}\right),$$
where at the right-hand-side we recognize the ``logsumexp`` operation applied to the vector ``loglik_vec``.

## WAIC Computation with arviz

arviz computes these quantity automatically:

In [ ]:
with beta_binomial:
    display(az.waic(trace_bb))

## WAIC Manual computation

* Write the likelihood and the log-likelihood function of the observations in closed form. Note: for these calculations, we cannot disregard multiplicative/additive terms not depending on $\theta$.

The likelihood is:
$$p(y | \theta) = {{n}\choose{y}} \theta^{y} \cdot (1-\theta)^{n-y}.$$

The log-likelihood is:
$$\log p(y | \theta) = \log {{n}\choose{y}} + y \log(\theta) + (n-y)\log(1-\theta).$$

* Obtain the likelihood and the log-likelihood as python functions (including multiplicative/additive factors)

In [ ]:
def lik(theta):
    return scipy.special.binom(n, y) * (theta ** y) * (1 - theta)**(n-y)

In [ ]:
def log_lik(theta):
    return np.log(scipy.special.binom(n, y)) + y*np.log(theta) + (n-y)*np.log(1 - theta)

* Extract the samples $\Theta$ from the trace.

In [ ]:
theta_trace = np.array(trace_bb.posterior.theta).ravel()
S = theta_trace.shape[0]

* Compute $\rm{lppd}$ and $p_{\rm waic}$

In [ ]:
log_lik_vec = log_lik(theta_trace)
# log_lik_pm = np.array(trace_bb.log_likelihood.y_obs).ravel()

In [ ]:
# lppd = np.log(np.mean(lik(theta_trace)))
lppd = scipy.special.logsumexp(log_lik_vec) - np.log(S) # sum is equivalent to logsumexp in log domain...

In [ ]:
p_waic = np.var(log_lik_vec) # correct
p_waic

* Compute $\rm{waic}$ and ${\rm elpd}$

In [ ]:
waic = -2*(lppd - p_waic); waic

In [ ]:
elpd = (lppd - p_waic); elpd